# Objective Testing

This notebook determines whether the robustness objective function using the toggling frame first order condition is correct. 

#### Imports

In [2]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../../QuantumCollocation.jl")
using Revise
using QuantumCollocation
using Piccolo
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using Statistics

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
   Resolving package versions...
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`


## 

#### Objective Testing

In [12]:
# Copy the FirstOrderObjective function
function FirstOrderObjective(
    H_err::AbstractMatrix{<:Number},
    traj::NamedTrajectory,
    times::AbstractVector{Int};
    Qs::AbstractVector{<:Float64}=fill(1.0, length(times))
)
    Ũ⃗_indices = [collect(slice(k, traj.components.Ũ⃗, traj.dim)) for k=1:traj.T]
        
    function ℓ(Z::AbstractVector{<:Real})
        Ũ⃗s = [Z[idx] for idx in Ũ⃗_indices]
        Us = [iso_vec_to_operator(Ũ⃗) for Ũ⃗ in Ũ⃗s]
        terms = [U' * H_err * U for U in Us]
        sum_terms = sum(terms)
        return norm(tr(sum_terms' * sum_terms), 1) / real(traj.T * norm(H_err, 1))
    end

    ∇ℓ = Z -> ForwardDiff.gradient(ℓ, Z)

    function ∂²ℓ_structure()
        Z_dim = traj.dim * traj.T + traj.global_dim
        structure = spzeros(Z_dim, Z_dim)
        all_Ũ⃗_indices = vcat(Ũ⃗_indices...)
        
        for i in all_Ũ⃗_indices
            for j in all_Ũ⃗_indices
                structure[i, j] = 1.0
            end
        end
        
        structure_pairs = collect(zip(findnz(structure)[1:2]...))
        return structure_pairs
    end

    function ∂²ℓ(Z::AbstractVector{<:Real})
        structure_pairs = ∂²ℓ_structure()
        H_full = ForwardDiff.hessian(ℓ, Z)
        ∂²ℓ_values = [H_full[i, j] for (i, j) in structure_pairs]
        
        return ∂²ℓ_values
    end

    return Objective(ℓ, ∇ℓ, ∂²ℓ, ∂²ℓ_structure)
end

FirstOrderObjective (generic function with 1 method)

In [4]:
# Problem parameters
T = 50
Δt = 0.2
U_goal = GATES.H
H_drives_test = [PAULIS.X, PAULIS.Y]
Hₑ = 0.1 * PAULIS.Z # error Hamiltonian
rob_scale = 1 / 8.0
piccolo_opts = PiccoloOptions(verbose=false)

PiccoloOptions(false, true, ExponentialAction.expv, true, false, nothing, 1.0, false, false, 0.01, 0.01)

In [5]:
sys_test = QuantumSystem(H_drives_test)
# Create a trajectory optimization problem
test_prob = UnitarySmoothPulseProblem(
    sys_test, U_goal, T, Δt;
    H_err = Hₑ,
    activate_rob_loss=true,
    piccolo_options=piccolo_opts
)

solve!(test_prob, max_iter=50, print_level=0)

# Extract trajectory data
Z_vec = vec(test_prob.trajectory)
println("Trajectory vector size: ", size(Z_vec))
println("Trajectory components: ", keys(test_prob.trajectory.components))
println("Trajectory dimensions: ", test_prob.trajectory.dim)
println("Trajectory time steps: ", test_prob.trajectory.T)


println("\nTesting FirstOrderObjective function...")

    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Trajectory vector size: (750,)
Trajectory components: (:Ũ⃗, :a, :da, :dda, :Δt)
Trajectory dimensions: 15
Trajectory time steps: 50

Testing FirstOrderObjective function...


In [13]:
# Test the FirstOrderObjective function
obj = FirstOrderObjective(Hₑ, test_prob.trajectory, [T])

# Evaluate the objective function
obj_value = obj.L(Z_vec)
println("Objective value: ", obj_value)

# Evaluate the gradient
grad_value = obj.∇L(Z_vec)
println("Gradient size: ", size(grad_value))
println("Gradient norm: ", norm(grad_value))
println("First 10 gradient elements: ", grad_value[1:min(10, length(grad_value))])

# Test Hessian structure
hess_structure = obj.∂²L_structure()
println("\nHessian structure pairs: ", length(hess_structure))
println("First 5 structure pairs: ", hess_structure[1:min(5, length(hess_structure))])

# Evaluate Hessian values
hess_values = obj.∂²L(Z_vec)
println("Hessian values size: ", size(hess_values))
println("Hessian values norm: ", norm(hess_values))
println("First 5 Hessian values: ", hess_values[1:min(5, length(hess_values))])

Objective value: 0.0018878032614524968
Gradient size: (750,)
Gradient norm: 0.03886185573303267
First 10 gradient elements: [0.0009926963666354583, 0.0026804618658965176, 0.0, 0.0026328889457729195, -0.0026804618658965176, 0.0009926963666254487, 0.0026328889457729195, 0.0, 0.0, 0.0]

Hessian structure pairs: 160000
First 5 structure pairs: [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]
Hessian values size: (160000,)
Hessian values norm: 0.8037673830242641
First 5 Hessian values: [0.00899269636663546, 0.0, 0.0, 0.0, -0.0026804618658965176]


In [14]:
println("\nVerifying gradient computation...")

# Direct ForwardDiff gradient
grad_direct = ForwardDiff.gradient(obj.L, Z_vec)
grad_obj = obj.∇L(Z_vec)

# Compare gradients
grad_diff = norm(grad_direct - grad_obj)
println("Gradient difference norm: ", grad_diff)
println("Gradients match: ", isapprox(grad_direct, grad_obj, rtol=1e-10))

# Verify Hessian using ForwardDiff
println("\nVerifying Hessian computation...")

# Direct ForwardDiff Hessian
hess_direct = ForwardDiff.hessian(obj.L, Z_vec)
println("Direct Hessian size: ", size(hess_direct))

# Convert structured Hessian back to full matrix for comparison
hess_structure = obj.∂²L_structure()
hess_values = obj.∂²L(Z_vec)
hess_sparse = sparse([p[1] for p in hess_structure], [p[2] for p in hess_structure], hess_values, length(Z_vec), length(Z_vec))
hess_full = Matrix(hess_sparse)

# Compare Hessians (only non-zero elements)
hess_diff_norm = 0.0
for (i, (row, col)) in enumerate(hess_structure)
    hess_diff_norm += (hess_direct[row, col] - hess_values[i])^2
end
hess_diff_norm = sqrt(hess_diff_norm)

println("Hessian difference norm (structured elements): ", hess_diff_norm)
println("Hessians match: ", hess_diff_norm < 1e-10)

println("\nFirstOrderObjective function test completed successfully!")


Verifying gradient computation...
Gradient difference norm: 0.0
Gradients match: true

Verifying Hessian computation...
Direct Hessian size: (750, 750)
Hessian difference norm (structured elements): 0.0
Hessians match: true

FirstOrderObjective function test completed successfully!
